In [7]:
#Imports

import pandas as pd
import numpy as np

In [166]:
#Functions

def sigmoid(z):
    g = np.divide(1, (1 + np.exp(-z)))
    return g

def costFunction(theta, X, y):
    m = y.size
    
    h = sigmoid(np.matmul(X, theta))
    
    J = (1 / m) * (np.matmul(-y.T, np.log(h)) - np.matmul((1 - y.T), np.log(1 - h)))
    return J
    
def gradientDescent(m, h, y, X):
    gradient = np.zeros(theta.size)
    gradient = (1 / m) * (np.transpose(h - y) * X)
    return gradient

In [167]:
train_df = pd.read_csv("train.csv")

#Drop columns and replace female -> 0, male -> 1
train_df = train_df.drop(["Name", "Ticket", "Embarked", "Cabin"], axis=1)
train_df = train_df.replace("female", 0)
train_df = train_df.replace("male", 1)

X = train_df.values
y = train_df.iloc[:, -1:]

#m = no. of examples; n = no. of features
m, n = X.shape

#Add the intercept term / bias
bias = np.ones(m, order='F')
X = np.insert(X, 0, bias, axis=1)

#Initial fitting
initial_theta = np.zeros((n + 1, 1))

#Compute the cost and gradient with initial values
cost = costFunction(initial_theta, X, y)
#gradient = gradientDescent(m, sigmoid(X * initial_theta), y, X)

ValueError: Shape of passed values is (1, 1), indices imply (1, 891)